# Ajuste de isocronas

Para ajustar las isocronas teóricas, primero se descargan los datos desde la base de datos de PARSEC usando un rango en metalicidad y edad de acuerdo al cúmulo que se quiere estudiar.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

Primero, se leen todas las isocronas que se descargaron y se quita filterslambdas.dat de la lista para evitar problemas.

In [ ]:
isochrones = glob.glob("/home/steven/Documents/IsocronasFit/isochrones_blanco1/gaiaedr3/*.dat")
isochrones.remove('/home/steven/Documents/IsocronasFit/isochrones_blanco1/gaiaedr3/filterslambdas.dat')

def open_file(x):
    y = pd.read_csv(x, delim_whitespace=True, skiprows=9)
    return y

def change_columns(x):
    y = x.rename(columns={'#':'Zini', 'Zini':'MH', 'MH':'logAge','logAge':'Mini','Mini':'int_IMF',
                          'int_IMF':'Mass','Mass':'logL','logL':'logTe','logTe':'logg','logg':'label',
                          'label':'mbolmag','mbolmag':'Gmag','Gmag':'G_BPmag','G_BPmag':'G_RPmag',
                          'G_RPmag':'delete'})
    return y

def remove_columns(x):
    y = x.drop('delete', axis=1)
    return y

def remove_rows(x):
    y = x.dropna(axis=0, subset=['Gmag'])
    return y


#isoc_data = pd.concat(map(open_file, isochrones[151:204]), ignore_index=True)
isoc_data = pd.concat(map(open_file, isochrones), ignore_index=True)
isoc_data = remove_rows(remove_columns(change_columns(isoc_data)))
#isoc_data.to_csv('./isoc/isoc_data.csv')
print(isoc_data.head())

Es necesario ordenar los archivos, pues todos vienen en archivos separados. Para esto, se concatenan y se ordenar por rango de metalicidad y edad.

Con los archivos ordenados, se seleccionan solo los datos de interés como Zini, logAge, Gmag y BP_RP, y se crea un .csv con solo estos datos. Es necesario eliminar los NA usando la función dropna().

In [ ]:
data = isoc_data #pd.read_csv('./isoc/isoc_data.csv')
data['Zini'] = pd.to_numeric(data['Zini'],errors='coerce')
data['logAge'] = pd.to_numeric(data['logAge'],errors='coerce')
data['Gmag'] = pd.to_numeric(data['Gmag'],errors='coerce')
data['G_BPmag'] = pd.to_numeric(data['G_BPmag'],errors='coerce')
data['G_RPmag'] = pd.to_numeric(data['G_RPmag'],errors='coerce')
#data['bp_rp'] = data['G_BPmag'] - data['G_RPmag'] 
data['BP_RP'] = data['G_BPmag'] - data['G_RPmag']
data = data.dropna(axis=0,subset=['Gmag','G_BPmag','G_RPmag'])
#data = data[data['Gmag'].apply(lambda x: str(x).isdigit())]

data = data[['Zini','logAge','Gmag','BP_RP']].to_csv('./isoc/consolidated_blanco1.csv')
#data.dropna(axis=0,subset=['Gmag','G_BPmag','G_RPmag'])[['Zini','logAge','Gmag','bp_rp']].to_csv('consolidated.csv')
#print(data.keys())
data

Hasta aquí, se ha generado un archivo que se llama consolidated_'cluster'.csv que tiene los datos de interés.

Si ya se tiene un consolidated_'cluster'.csv, no es necesario correr la parte de arriba.

# Encontrando la mejor isocrona calculando errores

Para calcular la mejor isocrona, se calculan los errores de la isocrona teórica respecto a los datos reales. Al final, se obtiene un .csv con el valor de metalicidad, la edad y su error. Se escoge la que menor error tenga.

In [3]:
df = pd.read_csv('/home/steven/Dropbox/Data/Field_data/blanco1_dbscan.csv')
#print(df['parallax'])

isochrones = pd.read_csv('./isoc/consolidated_blanco1.csv')
isochrones = isochrones[isochrones['Gmag'] > isochrones['BP_RP']]#*7.778068

z_rate = np.sort(isochrones['Zini'].unique())
log_age = np.sort(isochrones['logAge'].unique())

In [ ]:
save_z_rate = []
save_log_age = []
differ = []

g_mag = df['g'] + 5 - 5*np.log10(1/df['parallax'])
color = df['bp'] - df['rp']

for i in range(len(z_rate)):
    for j in range(len(log_age)):
        isoc_test = isochrones[(isochrones['Zini'] == z_rate[i]) & (isochrones['logAge'] == log_age[j])]
        isoc_test = np.array(isoc_test)
        for k in range(len(isoc_test)):
            diff = np.sqrt((color - isoc_test[k,-1])**2 + (g_mag - isoc_test[k,-2])**2)
        differ.append(min(diff))
        save_z_rate.append(z_rate[i])
        save_log_age.append(log_age[j])

In [ ]:
data = {'z_rate':save_z_rate,'log_age':save_log_age,'min_difference':differ}

data_f = pd.DataFrame(data)
data_f.to_csv('./isoc/results_blanco1_dbscan.csv')

In [ ]:
data_result = pd.read_csv('./isoc/results_blanco1_dbscan.csv')
#data_result = data_dbscan[data_dbscan['error']>0.0]
print(data_result.sort_values(by='min_difference').head(30))

In [ ]:
test_isoc = np.array(data_result.sort_values(by='min_difference'))
test_isoc

In [ ]:
isochrones = pd.read_csv('./isoc/consolidated_blanco1.csv')
df = pd.read_csv('/home/steven/Dropbox/Data/Field_data/blanco1_dbscan.csv')

g_mag = df['g'] + 5 - 5*np.log10(1 / df['parallax'])
pos = 0
isochrones = isochrones[(isochrones['Zini']==test_isoc[pos][1]) & (isochrones['logAge']==test_isoc[pos][2])]

plt.figure(figsize=(5,5))
plt.plot(isochrones['BP_RP'], isochrones['Gmag'], label=r'z={} logAge={:.5}'.format(test_isoc[pos][1],test_isoc[pos][2]))
plt.scatter(df['bp'] - df['rp'], g_mag, color='black',s=5)
plt.gca().invert_yaxis()
plt.xlabel(r"$G_{BP}-G_{RP}$",fontsize=14)
plt.ylabel("G",fontsize=14)
plt.legend(fontsize=12,loc=0)
#plt.savefig('./isocrona_blanco1.png',bbox_inches='tight')

In [ ]:
#df = pd.read_csv('/home/steven/Dropbox/Data/Field_data/blanco1_dbscan.csv')
#df['parallax'] = 1000/df['parallax']

#isochrones = pd.read_csv('./isoc/consolidated_blanco1.csv')
#isochrones = isochrones[isochrones['Gmag'] > isochrones['BP_RP']]#*7.778068

#z_rate = np.sort(isochrones['Zini'].unique())
#log_age = np.sort(isochrones['logAge'].unique())

#global_errors=[]
#save_z_rate =[]
#save_log_age=[]
#print(np.shape(z_rate), np.shape(log_age))
#distance = np.median(df['parallax'])
#g_mag = df['g'] + 5 - 5*np.log10(np.median(df['parallax']))
#for i in range(len(z_rate)):
#    for j in range(len(log_age)):
#        error_isoc = 0
#        isoc_test = isochrones[(isochrones['Zini'] == z_rate[i]) & (isochrones['logAge'] == log_age[j])]
#        isoc_test = np.array(isoc_test)
#        for k in range(len(isoc_test)):
#            diff = abs((df['bp'] - df['rp']) - isoc_test[k,-1]).argmin()
#            error_isoc += (g_mag[diff] - isoc_test[k,-2])**2
#        global_errors.append(error_isoc)
#        #save_cluster.append(clusters_names[cd])
#        save_z_rate.append(z_rate[i])
#        save_log_age.append(log_age[j])
#differ = []
#g_mag = df['g'] + 5 - 5*np.log10(df['parallax'])
#for i, j in zip(range(len(z_rate)), range(len(log_age))):
#    error_isoc = 0
#    isoc_test = isochrones[(isochrones['Zini'] == z_rate[i]) & (isochrones['logAge'] == log_age[j])]
#    isoc_test = np.array(isoc_test)
#    for k in range(len(isoc_test)):
#        #diff = abs((df['bp'] - df['rp']) - isoc_test[k,-1]).argmin()
#        diff = np.sqrt(((df['bp'] - df['rp']) - isoc_test[k,-1])**2 + (g_mag - isoc_test[k,-2])**2).argmin()
#        differ.append(diff)
#        #if diff < 15:
#        error_isoc += (g_mag[diff] - isoc_test[k,-2])**2
            
#    global_errors.append(error_isoc)
#    save_z_rate.append(z_rate[i])
#    save_log_age.append(log_age[j])
    
#data = {'z_rate':save_z_rate,'log_age':save_log_age,'error':global_errors}
 
# Create DataFrame
#df = pd.DataFrame(data)
#df.to_csv('./isoc/results_blanco1_dbscan.csv')